# Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras.layers import Dense,Embedding,Flatten,Input,Concatenate, Reshape, Dropout,merge,Activation,LSTM,GRU,SimpleRNN,Conv1D, MaxPooling1D, Flatten
from tensorflow.python.keras.layers import merge
import pandas as pd
import numpy as np
#! pip install categorical_embedder
#import categorical_embedder as ce

In [2]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('eda_traffic.csv')

# Display data types
print("Data Types:")
print(df.dtypes)


Data Types:
holiday                 object
temp                   float64
rain_1h                float64
snow_1h                float64
clouds_all             float64
weather_main            object
weather_description     object
date_time               object
traffic_volume         float64
Day_Week                 int64
Weekend                  int64
clouds_bin              object
temp_interval           object
dtype: object


In [3]:
df

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,Day_Week,Weekend,clouds_bin,temp_interval
0,NaN,15.13,0.0,0.0,40.0,Clouds,scattered clouds,2012-10-02 09:00:00,5545.0,1,0,21-40,10 to 19
1,NaN,16.21,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 10:00:00,4516.0,1,0,61-80,10 to 19
2,NaN,16.43,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 11:00:00,4767.0,1,0,81-100,10 to 19
3,NaN,16.98,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 12:00:00,5026.0,1,0,81-100,10 to 19
4,NaN,17.99,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 13:00:00,4918.0,1,0,61-80,10 to 19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52546,NaN,10.30,0.0,0.0,75.0,Clouds,broken clouds,2018-09-30 19:00:00,3543.0,6,1,61-80,10 to 19
52547,NaN,9.61,0.0,0.0,90.0,Clouds,overcast clouds,2018-09-30 20:00:00,2781.0,6,1,81-100,0 to 9
52548,NaN,9.58,0.0,0.0,90.0,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159.0,6,1,81-100,0 to 9
52549,NaN,8.94,0.0,0.0,90.0,Clouds,overcast clouds,2018-09-30 22:00:00,1450.0,6,1,81-100,0 to 9


In [4]:
df.shape

(52551, 13)

# Remove Duplicate Rows (4.2.2.3.3)

In [5]:
# Drop duplicates and keep the first occurrence
df.drop_duplicates(keep='first', inplace=True)

# Outlier (4.2.2.3.5)
 1. Temperature
 2. Rain

In [6]:
filtered_df = df[df['temp'] < -30][['date_time', 'temp']]
filtered_df

,date_time,temp
11658,2014-01-31 03:00:00,-273.15
11659,2014-01-31 04:00:00,-273.15
11660,2014-01-31 05:00:00,-273.15
11661,2014-01-31 06:00:00,-273.15
11706,2014-02-02 03:00:00,-273.15
11707,2014-02-02 04:00:00,-273.15
11708,2014-02-02 05:00:00,-273.15
11709,2014-02-02 06:00:00,-273.15
11710,2014-02-02 07:00:00,-273.15
11711,2014-02-02 08:00:00,-273.15


In [7]:
# Create a new DataFrame 'abc' as a copy of the original DataFrame 'df'
abc = df.copy()

# Assuming 'abc' is your DataFrame containing temperature data
# Convert 'date_time' column to datetime format if needed
abc['date_time'] = pd.to_datetime(abc['date_time'])

# Sort the DataFrame by 'date_time' to ensure sequential order
abc.sort_values(by='date_time', inplace=True)

# List of dates with outliers in temperature
dates_with_outliers = ['2014-01-31 03:00:00', '2014-01-31 04:00:00', '2014-01-31 05:00:00', '2014-01-31 06:00:00', '2014-02-02 03:00:00','2014-02-02 04:00:00','2014-02-02 05:00:00','2014-02-02 06:00:00','2014-02-02 07:00:00','2014-02-02 08:00:00']

# Iterate over each date in the list
for date in dates_with_outliers:
    # Find the index of the row with the specified date in the DataFrame 'abc'
    row_index = abc.index[abc['date_time'] == pd.to_datetime(date)]
    
    # Check if the row exists
    if len(row_index) > 0:
        # Get the index of the previous hour
        prev_hour_index = row_index[0] - 1
        
        # Check if the previous hour index is valid
        if prev_hour_index >= 0:
            # Replace the temperature value in DataFrame 'abc' with the value from the previous hour
            abc.at[row_index[0], 'temp'] = abc.at[prev_hour_index, 'temp']


In [8]:
abc['date_time'] = pd.to_datetime(abc['date_time'])

# Extract date from 'date_time' column
abc['date_only'] = abc['date_time'].dt.date

# Search for rows with a specific date
search_date = pd.to_datetime('2014-02-02').date() 
result_df = abc[abc['date_only'] == search_date]

result_df[['date_time', 'temp']]

,date_time,temp
11703,2014-02-02 00:00:00,-17.56
11704,2014-02-02 01:00:00,-17.67
11705,2014-02-02 02:00:00,-17.78
11706,2014-02-02 03:00:00,-17.78
11707,2014-02-02 04:00:00,-17.78
11708,2014-02-02 05:00:00,-17.78
11709,2014-02-02 06:00:00,-17.78
11710,2014-02-02 07:00:00,-17.78
11711,2014-02-02 08:00:00,-17.78
11712,2014-02-02 09:00:00,-17.53


In [9]:
# Define the extreme threshold for heavy rainfall
extreme_threshold = 31.3

# Loop through each date in the DataFrame
for date in abc['date_time']:
    # Find the index of the current date
    index = abc.index[abc['date_time'] == date].tolist()[0]
    
    # Check if the rainfall value exceeds the extreme threshold
    if abc.loc[index, 'rain_1h'] > extreme_threshold:
        # Set the rainfall value to the extreme threshold
        abc.loc[index, 'rain_1h'] = extreme_threshold


In [10]:
# Rainfall greater than 30mm 

abc[abc['rain_1h'] > 30]

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,Day_Week,Weekend,clouds_bin,temp_interval,date_only
6362,NaN,22.67,31.3,0.0,76.0,Rain,very heavy rain,2013-06-24 11:00:00,4802.0,0,1,61-80,20 to 29,2013-06-24
7409,NaN,15.95,31.3,0.0,68.0,Rain,very heavy rain,2013-08-07 02:00:00,315.0,2,0,61-80,10 to 19,2013-08-07
33080,NaN,28.96,31.3,0.0,75.0,Rain,very heavy rain,2016-07-11 17:00:00,5535.0,0,1,61-80,20 to 29,2016-07-11
33944,NaN,24.56,31.3,0.0,0.0,Rain,very heavy rain,2016-08-16 17:00:00,4913.0,1,0,NaN,20 to 29,2016-08-16


# Missing Value ( 4.2.2.3.2 and 4.2.2.3.9 )

In [11]:

abc.isnull().sum()

holiday                51279
temp                   11976
rain_1h                11976
snow_1h                11976
clouds_all             11976
weather_main           11976
weather_description    11976
date_time                  0
traffic_volume         11976
Day_Week                   0
Weekend                    0
clouds_bin             13940
temp_interval          11986
date_only                  0
dtype: int64

In [12]:
# Define the date range for filtering
start_date = '2014-08-08'
end_date = '2015-07-01'

# Filter rows based on the date range
abc1 = abc[(abc['date_time'] <= start_date) | (abc['date_time'] >= end_date)]
# Drop columns 'clouds_bin', 'temp_interval', and 'date_only' from DataFrame 'abc1'
abc1.drop(columns=['clouds_bin', 'temp_interval', 'date_only'], inplace=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_4748\1294668505.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abc1.drop(columns=['clouds_bin', 'temp_interval', 'date_only'], inplace=True)


In [13]:
abc1.isnull().sum()

holiday                43432
temp                    4279
rain_1h                 4279
snow_1h                 4279
clouds_all              4279
weather_main            4279
weather_description     4279
date_time                  0
traffic_volume          4279
Day_Week                   0
Weekend                    0
dtype: int64

In [14]:
abc1.index = pd.to_datetime(abc1['date_time']) # Ensure the index is datetime

# Perform time-based interpolation and backward fill for missing values
abc1[['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'traffic_volume']] = abc1[['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'traffic_volume']].interpolate(method='time', axis=0, limit_direction='backward')

# Backfill missing categorical values in the 'weather_description' column
abc1['weather_description'] = abc1['weather_description'].fillna(method='bfill')
abc1['weather_main'] = abc1['weather_main'].fillna(method='bfill')

C:\Users\Hp\AppData\Local\Temp\ipykernel_4748\599509671.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abc1[['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'traffic_volume']] = abc1[['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'traffic_volume']].interpolate(method='time', axis=0, limit_direction='backward')
C:\Users\Hp\AppData\Local\Temp\ipykernel_4748\599509671.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abc1['weather_description'] = abc1['weather_description'].fillna(method='bfill')
C:\Users\Hp\

In [15]:
abc1.isnull().sum()

holiday                43432
temp                       0
rain_1h                    0
snow_1h                    0
clouds_all                 0
weather_main               0
weather_description        0
date_time                  0
traffic_volume             0
Day_Week                   0
Weekend                    0
dtype: int64

In [16]:
import pandas as pd

# Use .loc to set the value without triggering the warning
abc1.loc[abc1['date_time'] == '2016-01-18 00:00:00', 'holiday'] = 'Martin Luther King Jr Day'

In [17]:
# Find the indices of rows where the holiday is not "None" and is not NaN
ind = abc1[(abc1['holiday'] != "None") & (~abc1['holiday'].isna())].index

# Iterate over the indices
for idx in ind:
    # Extract the date part from the datetime
    date_val = abc1.loc[idx, 'date_time'].date()
    
    # Update the 'holiday' column for the entire day with the holiday value
    abc1.loc[abc1['date_time'].dt.date == date_val, 'holiday'] = abc1.loc[idx, 'holiday']

# Use .loc to fill missing values in the 'holiday' column without triggering the warning
abc1.loc[:, 'holiday'] = abc1['holiday'].fillna('None')

In [18]:
abc1.loc[:, 'holiday'] = abc1['holiday'].fillna('None')


In [19]:
abc1.isnull().sum()


holiday                0
temp                   0
rain_1h                0
snow_1h                0
clouds_all             0
weather_main           0
weather_description    0
date_time              0
traffic_volume         0
Day_Week               0
Weekend                0
dtype: int64

In [20]:
abc1.holiday.value_counts()

holiday
None                         43408
Columbus Day                   120
Veterans Day                   120
Thanksgiving Day               120
Christmas Day                  120
New Years Day                  120
Washingtons Birthday           120
Memorial Day                   120
Independence Day               120
State Fair                     120
Labor Day                      120
Martin Luther King Jr Day       96
Name: count, dtype: int64

In [21]:
abc1.select_dtypes('object').columns

Index(['holiday', 'weather_main', 'weather_description'], dtype='object')

In [22]:
abc1.to_csv('data_prep_traffic.csv', index=False)


# Label Encoding

# (1) Weather Description

In [23]:
abc1.loc[:, 'weather_description'] = abc1['weather_description'].map(lambda x: x.lower())

# Create a copy of abc1 to avoid SettingWithCopyWarning
abc2 = abc1.copy()
abc2['weather_desc'] = abc2['weather_description']


In [24]:
abc1.weather_description.unique()

array(['scattered clouds', 'broken clouds', 'overcast clouds',
       'sky is clear', 'few clouds', 'light rain',
       'light intensity drizzle', 'mist', 'fog', 'proximity shower rain',
       'moderate rain', 'drizzle', 'heavy intensity rain',
       'proximity thunderstorm', 'haze', 'heavy snow',
       'thunderstorm with light rain', 'snow', 'light rain and snow',
       'light intensity shower rain', 'squalls',
       'proximity thunderstorm with rain', 'thunderstorm with rain',
       'thunderstorm with heavy rain', 'heavy intensity drizzle',
       'very heavy rain', 'thunderstorm with light drizzle',
       'thunderstorm', 'light snow', 'smoke', 'freezing rain',
       'light shower snow', 'sleet',
       'proximity thunderstorm with drizzle'], dtype=object)

In [25]:
abc2.loc[abc2['weather_desc'].str.contains('thunderstorm'), 'weather_desc'] = 'thunderstorm'
abc2.loc[abc2['weather_desc'].str.contains('sky is clear'), 'weather_desc'] = 'clear'

weather_type = ['clouds', 'rain', 'snow', 'fog', 'mist', 'haze', 'drizzle']
for weather in weather_type:
    abc2.loc[abc2['weather_desc'].str.contains(weather), 'weather_desc'] = weather

abc2.loc[abc2['weather_desc'].str.contains('drizzle'), 'weather_desc'] = 'rain'
abc2.loc[abc2['weather_desc'].str.contains('smoke'), 'weather_desc'] = 'haze'

weather_types = ['squalls', 'sleet']
for weather in weather_types:
    abc2.loc[abc2['weather_desc'].str.contains(weather), 'weather_desc'] = 'other'


In [26]:
abc2.weather_desc.unique()

array(['clouds', 'clear', 'rain', 'mist', 'fog', 'thunderstorm', 'haze',
       'snow', 'other'], dtype=object)

In [27]:
# Define a list of weather conditions considered as bad weather
wea_bad = ['fog', 'thunderstorm', 'snow', 'other', 'rain', 'haze']

# Define a custom function to binarize the weather condition into 'Good Weather' and 'Bad Weather'
def weather_bin(df, col):
    if df[col].isin(wea_bad):
        return 'bad_weather'  # Return 'Bad Weather' if the weather condition is in 'wea_bad'
    else:
        return 'good_weather'  # Return 'Good Weather' if the weather condition is not in 'wea_bad'


# Use numpy.where to binarize the weather condition into 'Good Weather' and 'Bad Weather'
abc2['weather_bin'] = np.where(abc2['weather_desc'].isin(wea_bad), 'bad_weather', 'good_weather')

In [28]:
abc2.weather_bin.value_counts(normalize=True)

weather_bin
good_weather    0.772593
bad_weather     0.227407
Name: proportion, dtype: float64

# (2) Snow

In [29]:
abc3 = abc2.copy()

# Define a custom function to binarize the 'snow_1h' column
def discretize_snow1h(x):
    if x == 0:
        return 'no_snow'
    else:
        return 'snow'

# Apply the custom function to the 'snow_1h' column to create a new column 'snow_status'
abc3['snow_bin'] = abc3['snow_1h'].apply(discretize_snow1h)
abc3.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,Day_Week,Weekend,weather_desc,weather_bin,snow_bin
date_time,,,,,,,,,,,,,,
2012-10-02 09:00:00,None,15.13,0.0,0.0,40.0,Clouds,scattered clouds,2012-10-02 09:00:00,5545.0,1,0,clouds,good_weather,no_snow
2012-10-02 10:00:00,None,16.21,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 10:00:00,4516.0,1,0,clouds,good_weather,no_snow
2012-10-02 11:00:00,None,16.43,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 11:00:00,4767.0,1,0,clouds,good_weather,no_snow
2012-10-02 12:00:00,None,16.98,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 12:00:00,5026.0,1,0,clouds,good_weather,no_snow
2012-10-02 13:00:00,None,17.99,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 13:00:00,4918.0,1,0,clouds,good_weather,no_snow


In [30]:
abc3.snow_bin.value_counts(normalize=True)

snow_bin
no_snow    0.998814
snow       0.001186
Name: proportion, dtype: float64

In [31]:
#abc3.drop(columns='snow_1h').to_csv('traffic_snow_bin.csv')

# (3) Holiday

In [32]:
abc4 = abc3.copy()

# Define a custom function to discretize the 'holiday' column into binary values
def discretize_holiday(x):
    if x == 'None':
        return 0  # Assign 0 for 'None' (No Holiday)
    else:
        return 1  # Assign 1 for other values (Holiday)

abc4['holi'] = abc4['holiday'].apply(discretize_holiday)

abc4['holi'].value_counts()


holi
0    43408
1     1296
Name: count, dtype: int64

In [33]:
abc4

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,Day_Week,Weekend,weather_desc,weather_bin,snow_bin,holi
date_time,,,,,,,,,,,,,,,
2012-10-02 09:00:00,None,15.13,0.0,0.0,40.0,Clouds,scattered clouds,2012-10-02 09:00:00,5545.0,1,0,clouds,good_weather,no_snow,0
2012-10-02 10:00:00,None,16.21,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 10:00:00,4516.0,1,0,clouds,good_weather,no_snow,0
2012-10-02 11:00:00,None,16.43,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 11:00:00,4767.0,1,0,clouds,good_weather,no_snow,0
2012-10-02 12:00:00,None,16.98,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 12:00:00,5026.0,1,0,clouds,good_weather,no_snow,0
2012-10-02 13:00:00,None,17.99,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 13:00:00,4918.0,1,0,clouds,good_weather,no_snow,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-09-30 19:00:00,None,10.30,0.0,0.0,75.0,Clouds,broken clouds,2018-09-30 19:00:00,3543.0,6,1,clouds,good_weather,no_snow,0
2018-09-30 20:00:00,None,9.61,0.0,0.0,90.0,Clouds,overcast clouds,2018-09-30 20:00:00,2781.0,6,1,clouds,good_weather,no_snow,0
2018-09-30 21:00:00,None,9.58,0.0,0.0,90.0,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159.0,6,1,thunderstorm,bad_weather,no_snow,0


# Weather Description(9) / Snow(2)/ Weather(2) label Encode

In [34]:
# Import the LabelEncoder from the preprocessing module in scikit-learn
from sklearn import preprocessing

# Create a LabelEncoder object
le = preprocessing.LabelEncoder()

abc5 = abc4.copy()

col_lst = ['weather_desc', 'snow_bin', 'weather_bin']

# Label encode each column in col_lst
encoded_data = {}
for col in col_lst:
    abc5[col] = le.fit_transform(abc5[col])
    encoded_data[col] = dict(zip(le.transform(le.classes_), le.classes_))


# Display the DataFrame after label encoding
abc5


,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,Day_Week,Weekend,weather_desc,weather_bin,snow_bin,holi
date_time,,,,,,,,,,,,,,,
2012-10-02 09:00:00,None,15.13,0.0,0.0,40.0,Clouds,scattered clouds,2012-10-02 09:00:00,5545.0,1,0,1,1,0,0
2012-10-02 10:00:00,None,16.21,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 10:00:00,4516.0,1,0,1,1,0,0
2012-10-02 11:00:00,None,16.43,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 11:00:00,4767.0,1,0,1,1,0,0
2012-10-02 12:00:00,None,16.98,0.0,0.0,90.0,Clouds,overcast clouds,2012-10-02 12:00:00,5026.0,1,0,1,1,0,0
2012-10-02 13:00:00,None,17.99,0.0,0.0,75.0,Clouds,broken clouds,2012-10-02 13:00:00,4918.0,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-09-30 19:00:00,None,10.30,0.0,0.0,75.0,Clouds,broken clouds,2018-09-30 19:00:00,3543.0,6,1,1,1,0,0
2018-09-30 20:00:00,None,9.61,0.0,0.0,90.0,Clouds,overcast clouds,2018-09-30 20:00:00,2781.0,6,1,1,1,0,0
2018-09-30 21:00:00,None,9.58,0.0,0.0,90.0,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159.0,6,1,8,0,0,0


In [35]:
abc5.dtypes

holiday                        object
temp                          float64
rain_1h                       float64
snow_1h                       float64
clouds_all                    float64
weather_main                   object
weather_description            object
date_time              datetime64[ns]
traffic_volume                float64
Day_Week                        int64
Weekend                         int64
weather_desc                    int32
weather_bin                     int32
snow_bin                        int32
holi                            int64
dtype: object

In [36]:
# Define a function to print the mapping of encoded values to their original meanings
def print_label_mapping(encoded_data, col_name):
    print(f"{col_name}:")
    for encoded_value, original_meaning in encoded_data[col_name].items():
        print(f"  {encoded_value}: {original_meaning}")

# Print the mapping of encoded values to their original meanings for each column
for col in col_lst:
    print_label_mapping(encoded_data, col)

weather_desc:
  0: clear
  1: clouds
  2: fog
  3: haze
  4: mist
  5: other
  6: rain
  7: snow
  8: thunderstorm
snow_bin:
  0: no_snow
  1: snow
weather_bin:
  0: bad_weather
  1: good_weather


In [37]:
abc5.to_csv('data_prep_bin_traffic.csv', index=False)